# Collecting Text Data using Twitter API

In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "kiiqIyYDM1LIpW1niRszMSVDE"
consumer_secret = "LxNoYOvbCocnTTb2oQnWFZgxQlDmgxFaG0jYf5xmXXYSS8s4YO"
access_token = "1245903768635322374-KQLSbIjZui9xcsAMBHaMs8mR5MWvsO"
access_token_secret = "4TIZboocLKBDrjGrPbMur6uc0Osk1ZhduefykBUAUQ01B"

In [4]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [5]:
query = "schools open in Maharashtra"

In [6]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x7f2c0552dc18>, _json={'created_at': 'Sat Nov 21 02:15:19 +0000 2020', 'id': 1329971624196534272, 'id_str': '1329971624196534272', 'full_text': "Maharashtra: Schools won't open on Monday, to remain shut till year-end https://t.co/a5Q7hCen5W", 'truncated': False, 'display_text_range': [0, 95], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/a5Q7hCen5W', 'expanded_url': 'http://toi.in/wmCyJa38', 'display_url': 'toi.in/wmCyJa38', 'indices': [72, 95]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://www.toi.in" rel="nofollow">cmssocialservice</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 30846824, 'id_str': '30846824', 'name': 'TOI Cities', 'screen_name': 'TOICitiesNews', 'location': '', 'description': 'Unmatched coverage fr

# Getting the Tweets and its attributes

In [7]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [8]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [9]:
stream(data=[ "schools open in Maharashtra"], file_name='my_tweets')

In [10]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @TOIMumbai: Maharashtra: Schools won't open...,Reshma Mutha,426,28,"Mumbai, India",False,0,4,2020-11-21 16:41:48
1,RT @TOIMumbai: Maharashtra: Schools won't open...,Laxman Sharma,59291,5255,India,False,0,4,2020-11-21 02:37:14
2,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:15:19
3,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:14:49
4,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:14:18


In [11]:
!pip install textblob

In [12]:
from textblob import TextBlob

In [13]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [14]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [15]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [16]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,RT @TOIMumbai: Maharashtra: Schools won't open...,Reshma Mutha,426,28,"Mumbai, India",False,0,4,2020-11-21 16:41:48,RT Maharashtra Schools won t open on Monday to...
1,RT @TOIMumbai: Maharashtra: Schools won't open...,Laxman Sharma,59291,5255,India,False,0,4,2020-11-21 02:37:14,RT Maharashtra Schools won t open on Monday to...
2,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:15:19,Maharashtra Schools won t open on Monday to re...
3,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:14:49,Maharashtra Schools won t open on Monday to re...
4,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:14:18,Maharashtra Schools won t open on Monday to re...


In [17]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @TOIMumbai: Maharashtra: Schools won't open...,Reshma Mutha,426,28,"Mumbai, India",False,0,4,2020-11-21 16:41:48,RT Maharashtra Schools won t open on Monday to...,Neutral
1,RT @TOIMumbai: Maharashtra: Schools won't open...,Laxman Sharma,59291,5255,India,False,0,4,2020-11-21 02:37:14,RT Maharashtra Schools won t open on Monday to...,Neutral
2,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:15:19,Maharashtra Schools won t open on Monday to re...,Neutral
3,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:14:49,Maharashtra Schools won t open on Monday to re...,Neutral
4,"Maharashtra: Schools won't open on Monday, to ...",TOI Cities,818382,21524,,True,0,0,2020-11-21 02:14:18,Maharashtra Schools won t open on Monday to re...,Neutral


In [18]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @TOIMumbai: Maharashtra: Schools won't open on Monday, to remain shut till year-end https://t.co/j3VEtC5AKI

Clean tweet:
 RT Maharashtra Schools won t open on Monday to remain shut till year end

Sentiment of the tweet:
 Neutral


In [19]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @indiatvnews: All schools in Mumbai to remain shut till December 31, says BMC  | via @IndiaTVNews 

#Mumbai #MumbaiSchools #BMC

https:/…

Clean tweet:
 RT All schools in Mumbai to remain shut till December 31 says BMC via Mumbai MumbaiSchools BMC https

Sentiment of the tweet:
 Neutral


In [22]:
n = 25
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @theicai Good Afternoon Sir
I need clarification on exams 
as in following 3 states schools &amp; educational instituti… https://t.co/y3c4RT9BDA

Clean tweet:
 Good Afternoon Sir I need clarification on exams as in following 3 states schools amp educational instituti

Sentiment of the tweet:
 Positive


In [23]:
df[df.Sentiment == 'Positive'].shape[0]

10

In [24]:
df[df.Sentiment == 'Neutral'].shape[0]

23

In [25]:
df[df.Sentiment == 'Negative'].shape[0]

2